## 参考资料：

* [Word2Vec代码实现](https://blog.csdn.net/qq_34290470/article/details/102843763)


In [17]:
import numpy as np
from collections import defaultdict

In [18]:
class word2vec():
 
    def __init__(self):
        self.n = settings['n']
        self.lr = settings['learning_rate']
        self.epochs = settings['epochs']
        self.window = settings['window_size']
 
    def generate_training_data(self, settings, corpus):
        """
        得到训练数据
        """
        #defaultdict(int)  一个字典，当所访问的键不存在时，用int类型实例化一个默认值
        word_counts = defaultdict(int)
 
        #遍历语料库corpus
        for row in corpus:
            for word in row:
                #统计每个单词出现的次数
                word_counts[word] += 1
 
        # 词汇表的长度
        self.v_count = len(word_counts.keys())
        # 在词汇表中的单词组成的列表
        self.words_list = list(word_counts.keys())
        # 以词汇表中单词为key，索引为value的字典数据
        self.word_index = dict((word, i) for i, word in enumerate(self.words_list))
        #以索引为key，以词汇表中单词为value的字典数据
        self.index_word = dict((i, word) for i, word in enumerate(self.words_list))
 
        training_data = []
 
        for sentence in corpus:
            sent_len = len(sentence)
 
            for i, word in enumerate(sentence):
 
                w_target = self.word2onehot(sentence[i])
 
                w_context = []
 
                for j in range(i - self.window, i + self.window):
                    if j != i and j <= sent_len - 1 and j >= 0:
                        w_context.append(self.word2onehot(sentence[j]))
 
                training_data.append([w_target, w_context])
 
        return np.array(training_data)
 
    def word2onehot(self, word):
 
        #将词用onehot编码
 
        word_vec = [0 for i in range(0, self.v_count)]
 
        word_index = self.word_index[word]
 
        word_vec[word_index] = 1
 
        return word_vec
 
    def train(self, training_data):
        
        #随机化参数w1,w2
        self.w1 = np.random.uniform(-1, 1, (self.v_count, self.n))
 
        self.w2 = np.random.uniform(-1, 1, (self.n, self.v_count))
 
        for i in range(self.epochs):
 
            self.loss = 0
 
            # w_t 是表示目标词的one-hot向量
            #w_t -> w_target,w_c ->w_context
            for w_t, w_c in training_data:
 
                #前向传播
                y_pred, h, u = self.forward(w_t)
 
                #计算误差
                EI = np.sum([np.subtract(y_pred, word) for word in w_c], axis=0)
 
                #反向传播，更新参数
                self.backprop(EI, h, w_t)
 
                #计算总损失
                self.loss += -np.sum([u[word.index(1)] for word in w_c]) + len(w_c) * np.log(np.sum(np.exp(u)))
 
            print('Epoch:', i, "Loss:", self.loss)
 
    def forward(self, x):
        """
        前向传播
        """
 
        h = np.dot(self.w1.T, x)
 
        u = np.dot(self.w2.T, h)
 
        y_c = self.softmax(u)
 
        return y_c, h, u
 
 
    def softmax(self, x):
        """
        """
        e_x = np.exp(x - np.max(x))
 
        return e_x / np.sum(e_x)
 
 
    def backprop(self, e, h, x):
 
        d1_dw2 = np.outer(h, e)
        d1_dw1 = np.outer(x, np.dot(self.w2, e.T))
 
        self.w1 = self.w1 - (self.lr * d1_dw1)
        self.w2 = self.w2 - (self.lr * d1_dw2)
 
    def word_vec(self, word):
 
        """
        获取词向量
        通过获取词的索引直接在权重向量中找
        """
 
        w_index = self.word_index[word]
        v_w = self.w1[w_index]
 
        return v_w
 
    def vec_sim(self, word, top_n):
        """
        找相似的词
        """
 
        v_w1 = self.word_vec(word)
        word_sim = {}
 
        for i in range(self.v_count):
            v_w2 = self.w1[i]
            theta_sum = np.dot(v_w1, v_w2)
 
            #np.linalg.norm(v_w1) 求范数 默认为2范数，即平方和的二次开方
            theta_den = np.linalg.norm(v_w1) * np.linalg.norm(v_w2)
            theta = theta_sum / theta_den
 
            word = self.index_word[i]
            word_sim[word] = theta
 
        words_sorted = sorted(word_sim.items(), key=lambda kv: kv[1], reverse=True)
 
        for word, sim in words_sorted[:top_n]:
            print(word, sim)
 
    def get_w(self):
        w1 = self.w1
        return  w1

In [19]:
#超参数
settings = {
    'window_size': 2,   #窗口尺寸 m
    #单词嵌入(word embedding)的维度,维度也是隐藏层的大小。
    'n': 10,
    'epochs': 50,         #表示遍历整个样本的次数。在每个epoch中，我们循环通过一遍训练集的样本。
    'learning_rate':0.01 #学习率
}

In [20]:
#数据准备
text = "natural language processing and machine learning is fun and exciting"
#按照单词间的空格对我们的语料库进行分词
corpus = [[word.lower() for word in text.split()]]
print(corpus)

[['natural', 'language', 'processing', 'and', 'machine', 'learning', 'is', 'fun', 'and', 'exciting']]


In [21]:
#初始化一个word2vec对象
w2v = word2vec()
 
training_data = w2v.generate_training_data(settings,corpus)

C:\Users\songjiahui\AppData\Local\Temp\ipykernel_5088\2069617963.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(training_data)


In [22]:
#训练
w2v.train(training_data)

Epoch: 0 Loss: 71.5441312379008
Epoch: 1 Loss: 68.3566135407369
Epoch: 2 Loss: 65.70471689086007
Epoch: 3 Loss: 63.50045450024576
Epoch: 4 Loss: 61.653006409757644
Epoch: 5 Loss: 60.08218052119147
Epoch: 6 Loss: 58.72388142505211
Epoch: 7 Loss: 57.529645339833266
Epoch: 8 Loss: 56.46371622681343
Epoch: 9 Loss: 55.49983707886061
Epoch: 10 Loss: 54.618591668856524
Epoch: 11 Loss: 53.80543325585156
Epoch: 12 Loss: 53.04929079653982
Epoch: 13 Loss: 52.34160034259834
Epoch: 14 Loss: 51.67563261219821
Epoch: 15 Loss: 51.04602246611997
Epoch: 16 Loss: 50.44843544042779
Epoch: 17 Loss: 49.879327808096235
Epoch: 18 Loss: 49.33577114799956
Epoch: 19 Loss: 48.81532201224998
Epoch: 20 Loss: 48.31592359917498
Epoch: 21 Loss: 47.835830495181604
Epoch: 22 Loss: 47.37355030250184
Epoch: 23 Loss: 46.92779781341851
Epoch: 24 Loss: 46.49745864106006
Epoch: 25 Loss: 46.08156007495282
Epoch: 26 Loss: 45.67924752686848
Epoch: 27 Loss: 45.2897653539527
Epoch: 28 Loss: 44.91244114734704
Epoch: 29 Loss: 44.546

In [23]:
# 获取词的向量
word = "machine"
vec = w2v.word_vec(word)
print(word, vec)

machine [-0.92233624 -0.15702451 -0.16708926 -0.97479067 -0.85454526 -0.4927798
 -0.30755922  0.84541473  0.17365586 -0.4731329 ]


In [24]:
 # 找相似的词
w2v.vec_sim("machine", 3)

machine 1.0
is 0.3194165697307216
learning 0.27548016668137326
